# 🌌 Spectral Affinity v3.0 — Music-CLAP Edition
### *The Ultimate AI Librarian for Semantic Clustering & Camelot-based Harmonic Flow*

---

| Module | Technology | Role |
|--------|-----------|------|
| 🧠 Neural Key Detection | nnAudio CQT1992v2 | Chromagram → Camelot notation |
| 🎵 Semantic Embeddings | Microsoft Music-CLAP | Vibe / genre / mood vectors |
| 🥁 Rhythm Analysis | librosa beat_track | BPM per track |
| 🔀 Semantic Clustering | KMeans (CLAP space) | Group tracks by acoustic vibe |
| 🎛️ Harmonic Sequencing | Camelot Wheel Engine | DJ-grade BPM-ascending playlist flow |
| 📦 Export | shutil + zipfile | Renamed files + ZIP download |

> **This tool is a read-only AI Librarian.**  
> It analyses, classifies, renames and copies your files — it never alters audio samples.


In [ ]:
# ============================================================
# CELL 1 — Install Dependencies (Deep Recovery Mode)
# ============================================================
import subprocess, sys, os

def run(cmd):
    subprocess.check_call([sys.executable, '-m', 'pip', *cmd])

print('⏳ Performing deep dependency recovery...')

try:
    # 1. Limpieza total de NumPy para evitar el error 'numpy.char'
    run(['uninstall', '-y', 'numpy'])
    
    # 2. Instalacion limpia de dependencias core
    # Forzamos 1.26.4 porque es la ultima v1 estable para audio neural
    run(['install', '-q', '--no-warn-conflicts', 'numpy==1.26.4'])
    
    # 3. Instalacion de librerias de Spectral Affinity
    run(['install', '-q', '--no-warn-conflicts', 
         'nnAudio>=0.3.3', 
         'msclap>=1.3.3', 
         'librosa>=0.10.0', 
         'scikit-learn>=1.3.0', 
         'tqdm', 
         'soundfile'])

    print('\n✅ Installation complete!')
    print('⚠️  IMPORTANT: You MUST restart the kernel now (Run -> Restart Kernel)')
    print('   to load the corrected NumPy structure.')
    
except Exception as e:
    print(f'❌ Critical Error during install: {e}')

In [ ]:
# ============================================================
# CELL 2 — Imports & GPU Setup
# ============================================================
import os
import re
import shutil
import zipfile
import warnings
import math
from pathlib import Path
from typing import List, Dict, Optional, Tuple

import numpy as np
import torch
import librosa
import soundfile as sf
from tqdm.auto import tqdm
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

# nnAudio GPU spectrogram
from nnAudio.features import CQT1992v2

# Music-CLAP
from msclap import CLAP

warnings.filterwarnings('ignore')

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'🖥️  Device: {DEVICE}')
if DEVICE.type == 'cuda':
    print(f'   GPU: {torch.cuda.get_device_name(0)}')
    print(f'   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
else:
    print('⚠️  No GPU detected — analysis will run on CPU (slower but functional).')

In [ ]:
# ============================================================
# CELL 3 — Camelot Wheel Definitions & Harmonic Engine
# ============================================================

# ---------------------------------------------------------------------------
# Krumhansl-Schmuckler key-profile templates (24 keys: 12 major + 12 minor)
# ---------------------------------------------------------------------------
KS_MAJOR = np.array([6.35, 2.23, 3.48, 2.33, 4.38, 4.09,
                     2.52, 5.19, 2.39, 3.66, 2.29, 2.88])
KS_MINOR = np.array([6.33, 2.68, 3.52, 5.38, 2.60, 3.53,
                     2.54, 4.75, 3.98, 2.69, 3.34, 3.17])

# Pitch class names (starting from C)
PITCH_NAMES = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']

# ---------------------------------------------------------------------------
# Camelot Wheel mapping  key_string -> camelot_code
# Format: 'Note Mode' -> 'NNX'  (NN = 01-12, X = A/B)
#   A = minor (inner ring)   B = major (outer ring)
# ---------------------------------------------------------------------------
CAMELOT_MAP: Dict[str, str] = {
    # Minor keys  (A = inner ring)
    'Ab minor': '01A', 'G# minor': '01A',
    'Eb minor': '02A', 'D# minor': '02A',
    'Bb minor': '03A', 'A# minor': '03A',
    'F minor':  '04A',
    'C minor':  '05A',
    'G minor':  '06A',
    'D minor':  '07A',
    'A minor':  '08A',
    'E minor':  '09A',
    'B minor':  '10A',
    'F# minor': '11A', 'Gb minor': '11A',
    'C# minor': '12A', 'Db minor': '12A',
    # Major keys  (B = outer ring)
    'B major':  '01B',
    'F# major': '02B', 'Gb major': '02B',
    'Db major': '03B', 'C# major': '03B',
    'Ab major': '04B', 'G# major': '04B',
    'Eb major': '05B', 'D# major': '05B',
    'Bb major': '06B', 'A# major': '06B',
    'F major':  '07B',
    'C major':  '08B',
    'G major':  '09B',
    'D major':  '10B',
    'A major':  '11B',
    'E major':  '12B',
}

# Reverse map: camelot -> canonical key name (for display)
CAMELOT_REVERSE: Dict[str, str] = {v: k for k, v in CAMELOT_MAP.items() if 'b' not in k.split()[0] or k.split()[0] in ('Bb','Db','Eb','Gb','Ab')}


def key_to_camelot(pitch_class: int, is_major: bool) -> str:
    """Convert a detected pitch class (0=C..11=B) and mode to Camelot notation."""
    note = PITCH_NAMES[pitch_class]
    mode = 'major' if is_major else 'minor'
    key_str = f'{note} {mode}'
    return CAMELOT_MAP.get(key_str, '??')


def camelot_neighbours(code: str) -> List[str]:
    """
    Return the 3 safe harmonic neighbours of a Camelot code:
      1. Same position (identity)        e.g. 08A -> 08A
      2. ±1 step clockwise/anticlockwise e.g. 08A -> 07A, 09A
      3. Relative mode switch            e.g. 08A -> 08B
    """
    if not re.match(r'^\d{2}[AB]$', code):
        return []
    num  = int(code[:2])   # 1-12
    mode = code[2]         # A or B
    alt  = 'B' if mode == 'A' else 'A'

    prev_num = ((num - 2) % 12) + 1
    next_num = (num % 12) + 1

    fmt = lambda n, m: f'{n:02d}{m}'
    return [
        fmt(num,      mode),   # same
        fmt(prev_num, mode),   # -1 step
        fmt(next_num, mode),   # +1 step
        fmt(num,      alt),    # relative mode switch
    ]


def get_next_harmonic(current_code: str, candidates: List[Dict]) -> Optional[Dict]:
    """
    Given the current Camelot code and a list of candidate track dicts
    (each with 'camelot' and 'bpm' keys), return the best next track:
      - Must be a harmonic neighbour
      - Prefer ascending BPM within compatible keys
    Returns None if no harmonic match found.
    """
    neighbours = set(camelot_neighbours(current_code))
    compatible = [t for t in candidates if t.get('camelot', '??') in neighbours]
    if not compatible:
        return None
    # Among compatible, pick the one with BPM closest to current (ascending preference)
    compatible.sort(key=lambda t: t.get('bpm', 0))
    return compatible[0]


print('✅ Camelot engine loaded.')
print(f'   Wheel codes available: {len(set(CAMELOT_MAP.values()))}')
# Quick sanity-check
assert camelot_neighbours('08A') == ['08A', '07A', '09A', '08B'], 'Neighbour check failed'
print('   Sanity check passed: 08A → [08A, 07A, 09A, 08B]')

In [ ]:
# ============================================================
# CELL 4 — Audio Analysis Engine (CQT Key + librosa BPM)
# ============================================================

# Build the CQT transform module once (reused across all tracks)
_cqt_transform = CQT1992v2(
    sr=22050,
    hop_length=512,
    fmin=librosa.note_to_hz('C2'),
    n_bins=84,          # 7 octaves × 12 bins/octave
    bins_per_octave=12,
    output_format='Magnitude',
    verbose=False,
).to(DEVICE)


def detect_key_cqt(y: np.ndarray, sr: int) -> str:
    """
    Detect musical key using GPU CQT + Krumhansl-Schmuckler profile matching.

    Steps:
    1. Resample to 22050 Hz
    2. Compute CQT spectrogram on GPU
    3. Sum energy per pitch class (chroma vector)
    4. Correlate chroma against all 24 KS templates (12-step circular shift)
    5. Return best-matching key in Camelot notation
    """
    if sr != 22050:
        y = librosa.resample(y, orig_sr=sr, target_sr=22050)
        sr = 22050

    # Mono tensor on GPU
    y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        cqt_mag = _cqt_transform(y_tensor)  # (1, n_bins, T)

    # Average across time, then fold into 12 pitch classes
    cqt_mean = cqt_mag.squeeze(0).mean(dim=-1).cpu().numpy()  # (84,)
    chroma = np.zeros(12)
    for i in range(84):
        chroma[i % 12] += cqt_mean[i]

    # Normalize chroma
    chroma_norm = chroma / (chroma.sum() + 1e-8)

    # Correlate against all 24 key profiles (circular shifts of KS templates)
    best_score = -np.inf
    best_pc    = 0
    best_major = True

    for pc in range(12):
        shifted_major = np.roll(KS_MAJOR, -pc)
        shifted_minor = np.roll(KS_MINOR, -pc)

        score_major = np.corrcoef(chroma_norm, shifted_major / shifted_major.sum())[0, 1]
        score_minor = np.corrcoef(chroma_norm, shifted_minor / shifted_minor.sum())[0, 1]

        if score_major > best_score:
            best_score = score_major
            best_pc    = pc
            best_major = True
        if score_minor > best_score:
            best_score = score_minor
            best_pc    = pc
            best_major = False

    return key_to_camelot(best_pc, best_major)


def detect_bpm(y: np.ndarray, sr: int) -> float:
    """
    Compute BPM using librosa beat tracking.
    Clamps to [60, 200] BPM to handle half-time / double-time artefacts.
    """
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    bpm = float(tempo[0]) if hasattr(tempo, '__len__') else float(tempo)

    # Halve / double until in range
    while bpm > 200:
        bpm /= 2
    while bpm < 60:
        bpm *= 2

    return round(bpm, 1)


print('✅ Audio analysis engine ready.')

In [ ]:
# ============================================================
# CELL 5 — Music-CLAP Embedding Extractor
# ============================================================
# Model: microsoft/msclap  —  checkpoint: music_2023
# Downloads ~1 GB on first run, cached afterwards.

print('Loading Music-CLAP model (this may download ~1 GB on first run)...')
_clap_model = CLAP(version='music_2023', use_cuda=(DEVICE.type == 'cuda'))
print('✅ Music-CLAP model loaded.')


def get_clap_embedding(audio_path: str) -> np.ndarray:
    """
    Extract a 512-d semantic embedding for a single audio file using Music-CLAP.
    Returns a L2-normalized numpy vector.
    """
    emb = _clap_model.get_audio_embeddings([audio_path], resample=True)  # (1, 512)
    emb_np = emb.squeeze(0).cpu().numpy() if isinstance(emb, torch.Tensor) else np.array(emb).squeeze(0)
    # L2-normalize
    norm = np.linalg.norm(emb_np)
    return emb_np / (norm + 1e-8)


print('   CLAP embedding function ready. Output shape: (512,)')

In [ ]:
# ============================================================
# CELL 6 — Batch GPU Analysis Pipeline
# ============================================================
# Processes every audio file in INPUT_DIR and returns a list of track dicts.

SUPPORTED_EXTS = {'.wav', '.mp3', '.flac', '.ogg', '.aac', '.m4a'}


def scan_audio_files(directory: str) -> List[Path]:
    """Recursively find all supported audio files in directory."""
    root = Path(directory)
    files = [p for p in root.rglob('*') if p.suffix.lower() in SUPPORTED_EXTS]
    files.sort(key=lambda p: p.name.lower())
    return files


def analyse_track(path: Path) -> Optional[Dict]:
    """
    Full neural analysis of a single audio file.
    Returns a dict with keys: path, name, camelot, bpm, embedding.
    Returns None on error.
    """
    try:
        # Load at 22050 Hz mono (read-only)
        y, sr = librosa.load(str(path), sr=22050, mono=True)

        camelot = detect_key_cqt(y, sr)
        bpm     = detect_bpm(y, sr)
        emb     = get_clap_embedding(str(path))

        return {
            'path':      path,
            'name':      path.stem,
            'ext':       path.suffix,
            'camelot':   camelot,
            'bpm':       bpm,
            'embedding': emb,
        }
    except Exception as e:
        print(f'  ⚠️  Error analysing {path.name}: {e}')
        return None


def run_batch_analysis(input_dir: str, batch_size: int = 4) -> List[Dict]:
    """
    Analyse all audio files in input_dir in batches.
    Returns a list of track metadata dicts.
    batch_size: number of tracks to process per VRAM flush.
    """
    files = scan_audio_files(input_dir)
    if not files:
        raise ValueError(f'No supported audio files found in {input_dir}')

    print(f'\n🔍 Found {len(files)} audio file(s) in {input_dir}')
    print(f'   Batch size: {batch_size} | Device: {DEVICE}\n')

    results = []
    for i in tqdm(range(0, len(files), batch_size), desc='Analysing tracks'):
        batch = files[i:i + batch_size]
        for path in batch:
            tqdm.write(f'  → {path.name}')
            track = analyse_track(path)
            if track is not None:
                results.append(track)

        # Clear VRAM cache between batches
        if DEVICE.type == 'cuda':
            torch.cuda.empty_cache()

    print(f'\n✅ Analysis complete: {len(results)}/{len(files)} tracks processed successfully.')
    return results


print('✅ Batch analysis pipeline ready.')

In [ ]:
# ============================================================
# CELL 7 — KMeans Semantic Clustering
# ============================================================
# Groups tracks by their Music-CLAP embeddings into N_CLUSTERS vibes.


def cluster_tracks(tracks: List[Dict], n_clusters: int) -> Dict[int, List[Dict]]:
    """
    Apply KMeans clustering to CLAP embedding vectors.
    Returns a dict mapping cluster_id -> list of track dicts.

    If n_clusters > number of tracks, falls back to 1 cluster.
    """
    if not tracks:
        return {}

    n_clusters = min(n_clusters, len(tracks))
    embeddings = np.stack([t['embedding'] for t in tracks])  # (N, 512)

    print(f'\n🔀 Running KMeans clustering:')
    print(f'   Tracks     : {len(tracks)}')
    print(f'   Clusters   : {n_clusters}')
    print(f'   Embedding  : {embeddings.shape}')

    km = KMeans(
        n_clusters=n_clusters,
        n_init=20,
        max_iter=500,
        random_state=42,
    )
    labels = km.fit_predict(embeddings)

    clusters: Dict[int, List[Dict]] = {i: [] for i in range(n_clusters)}
    for track, label in zip(tracks, labels):
        track['cluster'] = int(label)
        clusters[int(label)].append(track)

    # Print summary
    print(f'\n   Cluster sizes:')
    for cid, members in sorted(clusters.items()):
        print(f'   Cluster {cid + 1:02d}: {len(members):3d} tracks')

    return clusters


print('✅ Clustering module ready.')

In [ ]:
# ============================================================
# CELL 8 — Harmonic Sequencer (Camelot + BPM Ascending Flow)
# ============================================================


def build_harmonic_sequence(tracks: List[Dict]) -> List[Dict]:
    """
    Arrange tracks within a cluster into a DJ-ready harmonic sequence:

    Algorithm:
    1.  Sort all tracks by BPM ascending as the base order.
    2.  Start from the lowest-BPM track.
    3.  Greedily pick the next track using get_next_harmonic() from the
        remaining pool — prioritising harmonic compatibility first,
        then ascending BPM.
    4.  If no harmonic match exists, pick the next unvisited track by BPM
        (graceful fallback — marked with a warning flag).

    Returns the ordered list of track dicts with a 'harmonic_ok' boolean.
    """
    if not tracks:
        return []

    pool = sorted(tracks, key=lambda t: t.get('bpm', 0))
    sequence: List[Dict] = []
    current_camelot = pool[0]['camelot']

    while pool:
        nxt = get_next_harmonic(current_camelot, pool)
        if nxt is not None:
            nxt['harmonic_ok'] = True
        else:
            # Graceful fallback: take lowest-BPM remaining
            nxt = pool[0]
            nxt['harmonic_ok'] = False

        sequence.append(nxt)
        current_camelot = nxt['camelot']
        pool.remove(nxt)

    return sequence


def sequence_all_clusters(clusters: Dict[int, List[Dict]]) -> Dict[int, List[Dict]]:
    """Apply harmonic sequencing to every cluster."""
    sequenced = {}
    for cid, tracks in clusters.items():
        sequenced[cid] = build_harmonic_sequence(tracks)
        fallbacks = sum(1 for t in sequenced[cid] if not t.get('harmonic_ok', True))
        print(f'  Cluster {cid + 1:02d}: {len(tracks):3d} tracks sequenced '
              f'| {fallbacks} BPM-only fallbacks')
    return sequenced


print('✅ Harmonic sequencer ready.')

In [ ]:
# ============================================================
# CELL 9 — Export: Rename, Copy & ZIP
# ============================================================


def _sanitize(name: str) -> str:
    """Remove filesystem-unsafe characters from a filename."""
    return re.sub(r'[\\/:*?"<>|]', '_', name)


def export_results(
    sequenced_clusters: Dict[int, List[Dict]],
    out_dir: str,
    zip_name: str = 'spectral_affinity_results.zip',
) -> str:
    """
    Copy and rename each track into structured output folders:

    out_dir/
      cluster_01_vibe/
        01 - [08A] 120BPM TrackName.wav
        02 - [09A] 122BPM TrackName.wav
        ...
      cluster_02_vibe/
        ...
      spectral_affinity_results.zip

    Returns the path to the generated ZIP file.
    Audio originals are NEVER modified — only copied.
    """
    out_root = Path(out_dir)
    out_root.mkdir(parents=True, exist_ok=True)

    cluster_dirs = []

    for cid, tracks in sorted(sequenced_clusters.items()):
        # Named folder: cluster_01_vibe, cluster_02_vibe...
        folder_name = f'cluster_{cid + 1:02d}_vibe'
        cluster_dir = out_root / folder_name
        cluster_dir.mkdir(parents=True, exist_ok=True)
        cluster_dirs.append(cluster_dir)

        print(f'\n📂 {folder_name} ({len(tracks)} tracks)')

        for idx, track in enumerate(tracks, start=1):
            camelot = track.get('camelot', '??')
            bpm     = int(round(track.get('bpm', 0)))
            orig    = _sanitize(track['name'])
            ext     = track['ext']
            flag    = '' if track.get('harmonic_ok', True) else ' ⚠️'

            new_name = f'{idx:02d} - [{camelot}] {bpm}BPM {orig}{ext}'
            dst      = cluster_dir / new_name

            shutil.copy2(str(track['path']), str(dst))  # preserves timestamps
            print(f'  {idx:02d}  [{camelot}] {bpm} BPM  {orig}{ext}{flag}')

    # ── Pack into a single ZIP ────────────────────────────────────────────────
    zip_path = out_root / zip_name
    print(f'\n📦 Packaging ZIP → {zip_path}')

    with zipfile.ZipFile(str(zip_path), 'w', zipfile.ZIP_STORED) as zf:
        for cdir in cluster_dirs:
            for f in sorted(cdir.iterdir()):
                arcname = f'{cdir.name}/{f.name}'
                zf.write(str(f), arcname)

    zip_size_mb = zip_path.stat().st_size / 1e6
    print(f'✅ ZIP created: {zip_path.name}  ({zip_size_mb:.1f} MB)')

    return str(zip_path)


print('✅ Export module ready.')

In [ ]:
# ============================================================
# CELL 10 — ⚙️ CONFIGURATION & MAIN EXECUTION
# ============================================================
# 👇 Edit ONLY the variables in this block.
# Everything else runs automatically.

# ─── USER CONFIGURATION ────────────────────────────────────
INPUT_DIR  = '/kaggle/input/datasets/danieldobles/slavic-songs'
OUT_DIR    = '/kaggle/working/HARMONIC_RESULTS'

# Number of semantic clusters (vibes).
# Rule of thumb: 1 cluster per ~20-30 tracks, or set manually.
N_CLUSTERS = 6

# Tracks processed per GPU memory flush
# Reduce to 2 if you hit CUDA OOM; increase to 8 on A100.
BATCH_SIZE = 4
# ──────────────────────────────────────────────────────────


def main():
    print('=' * 64)
    print('  🌌 Spectral Affinity v3.0 — Music-CLAP Edition')
    print('=' * 64)
    print(f'  INPUT  : {INPUT_DIR}')
    print(f'  OUTPUT : {OUT_DIR}')
    print(f'  CLUSTERS: {N_CLUSTERS}  |  BATCH: {BATCH_SIZE}')
    print('=' * 64)

    # ── STEP 1: Deep neural analysis ─────────────────────────────
    print('\n✨ STEP 1/3  Neural Analysis (Key + BPM + CLAP embeddings)')
    tracks = run_batch_analysis(INPUT_DIR, batch_size=BATCH_SIZE)

    if not tracks:
        print('🚫 No tracks analysed. Check INPUT_DIR path and audio formats.')
        return

    # Print analysis summary table
    print('\n  Track Metadata Summary:')
    print(f'  {"#":<4} {"File":<45} {"Key":>5} {"BPM":>6}')
    print(f'  {"-"*4} {"-"*45} {"-"*5} {"-"*6}')
    for i, t in enumerate(tracks, 1):
        name_short = t['name'][:44] + ("…" if len(t['name']) > 44 else "")
        print(f'  {i:<4} {name_short:<45} {t["camelot"]:>5} {t["bpm"]:>6.1f}')

    # ── STEP 2: Semantic clustering ───────────────────────────────
    print('\n✨ STEP 2/3  Semantic Clustering (KMeans on CLAP Space)')
    clusters = cluster_tracks(tracks, n_clusters=N_CLUSTERS)

    # ── STEP 3: Harmonic sequencing ───────────────────────────────
    print('\n✨ STEP 3/3  Harmonic Sequencing (Camelot Wheel + BPM Flow)')
    sequenced = sequence_all_clusters(clusters)

    # ── EXPORT: Copy, rename, ZIP ─────────────────────────────────
    print('\n📦 Exporting results...')
    zip_path = export_results(sequenced, out_dir=OUT_DIR)

    print('\n' + '=' * 64)
    print('  ✅ Spectral Affinity v3.0 complete!')
    print(f'  📂 Results: {OUT_DIR}')
    print(f'  📦 ZIP    : {zip_path}')
    print('  🎧 Your harmonic playlists are ready to download.')
    print('=' * 64)

    # Kaggle notebook: trigger file download panel
    try:
        from IPython.display import FileLink, display
        display(FileLink(zip_path.replace("/kaggle/working/", "")))
    except Exception:
        pass


# Run!
main()